In [22]:
!rm diff_sorp.h5

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

L = 10.0  # Length of the domain
T = 1.0   # Total simulation time
Nx = 100  # Number of spatial grid points
Nt = 201  # Number of time steps
dx = L / (Nx - 1)  # Spatial step size
dt = T / Nt        # Time step size

for o in range(1000):
    D = np.random.uniform(1, 100)
    Kd = abs(np.random.uniform(D - 10, D + 10)) % 100
    alpha = D / (1 + Kd) * dt / dx**2  # Stability parameter
    print(o, D, Kd)
    if alpha > 0.5:
        print("alpha > 0.5")
        continue
    # Initialize concentration array
    C = np.zeros(Nx)  # Initial concentration
    C[int(Nx / 4):int(Nx / 2)] = 1.0  # Initial condition (step function)

    # Boundary conditions (Dirichlet: fixed concentration at boundaries)
    C[0] = 0.0
    C[-1] = 0.0

    # Store all concentration data over time
    C_all = np.zeros((Nt, Nx))  # Shape: (time steps, spatial points)

    # Time-stepping loop
    for t in range(Nt):
        C_new = C.copy()
        for i in range(1, Nx - 1):
            C_new[i] = C[i] + alpha * (C[i + 1] - 2 * C[i] + C[i - 1])
        C = C_new

        # Apply boundary conditions at each time step
        C[0] = 0.0
        C[-1] = 0.0

        # Store the current concentration profile
        C_all[t, :] = C
    try:
      with h5py.File("diff_sorp.h5", "a") as f:
          g = f.create_group(f"{o}")
          g.create_dataset("D", data=D)
          g.create_dataset("Kd", data=Kd)
          g.create_dataset("data", data=C_all)
    except:
      print(o)

# Now C_all contains the concentration data for all time steps
# You can save it to a file or analyze it further
# np.save("concentration_data.npy", C_all)

0 6.704182602663666 3.9736384197139376
alpha > 0.5
1 82.44376791378382 92.0820718580985
2 56.798624402647334 49.18858039824791
alpha > 0.5
3 5.21114155738064 0.7724322611499037
alpha > 0.5
4 67.7111928831382 60.908830145402696
alpha > 0.5
5 89.87955332578295 92.86273255566363
6 97.8691636433072 95.5696479295634
7 14.229549638255556 4.927072256910749
alpha > 0.5
8 82.29396439565238 91.3079362932105
9 9.244470177029612 6.881355303055388
alpha > 0.5
10 77.85380487375146 84.40892924721611
11 90.2070600483768 89.60564997439897
12 12.787818112367123 16.283786396387157
13 18.69073806904477 15.103339124316808
alpha > 0.5
14 10.705786382369011 1.7917318408092626
alpha > 0.5
15 13.789908572351756 18.351636775323882
16 77.94371863343014 77.17063254426179
17 1.6641287021245943 7.544615515038997
18 18.12530015207264 9.975318137862015
alpha > 0.5
19 74.5442011689423 79.54714427829369
20 47.9588372470241 50.580930092617564
21 2.863388361874349 0.3046286450465576
alpha > 0.5
22 99.59520289099696 4.408

In [26]:
!ls -lh

total 93M
-rw-r--r-- 1 root root 1.4M Jan 30 13:51 a.gif
-rw-r--r-- 1 root root  92M Jan 30 14:05 diff_sorp.h5
drwxr-xr-x 1 root root 4.0K Jan 28 14:22 sample_data


In [24]:
C_all.shape

(201, 100)

In [27]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm
import h5py
def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)

In [28]:
visualize_burgers([i for i in range(100)], C_all, "a.gif")

100%|██████████| 201/201 [00:00<00:00, 2031.29it/s]


In [30]:
from scipy.stats import skewnorm

with h5py.File("diff_sorp.h5", "a") as f:
    for key in f.keys():
        u = f[key]["data"][:]
        noise = skewnorm.rvs(a=1, scale=0.2, size=u.shape)
        u_noise = u + noise
        f[key].create_dataset("noise", data=u_noise)